# SPARQL Playground

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/sparql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>


## requirements.txt

In [39]:
%%capture
!pip install -q minio
!pip install -q kglab
!pip install sparqlwrapper


## imports

In [62]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from geopandas import GeoDataFrame

## Remote SPARQL server

In [41]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [43]:
rp1 = """
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX unit: <http://www.opengis.net/def/uom/OGC/1.0/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

select ?s ?s2l13 ?buildingLocation where {
            ?s schema:additionalType "power"  .
            ?s schema:geo ?geo .

            ?geo schema:identifier ?geoid .
            ?geoid schema:name "s2Level13" .
            ?geoid schema:value ?s2l13 .

            ?geo geo:asWKT ?buildingLocation .

            BIND("POINT(-84.3555 41.3175)"^^geo:wktLiteral AS ?centroid)
            BIND(geof:buffer(?centroid, 8.04672, unit:kilometer) AS ?searchArea)
}
"""

ep = "http://132.249.238.155/repositories/wenokn_ohio"

df = get_sparql_dataframe(ep, rp1)
df.head(10)

,s,s2l13,buildingLocation
0,https://ufokn.org/id/urmi/dpk4bzkkghxv,9817229210593263616,POINT( -84.33606510000001 41.30531429999997 2....
1,https://ufokn.org/id/urmi/dpk4bzkkghxv,9817229210593263616,POINT( -84.33606510000001 41.30531429999997 2....
2,https://ufokn.org/id/urmi/dpk4bzkkghxv,9817229210593263616,POINT( -84.33606510000001 41.30531429999997 2....
3,https://ufokn.org/id/urmi/dpk4bzkkghxv,9817229210593263616,POINT( -84.33606510000001 41.30531429999997 2....
4,https://ufokn.org/id/urmi/dpk4b8xu90h7,9817233883517681664,POINT( -84.3423355 41.26816799999996 8.0601425...
5,https://ufokn.org/id/urmi/dpk4b8xu90h7,9817233883517681664,POINT( -84.3423355 41.26816799999996 8.0601425...
6,https://ufokn.org/id/urmi/dpk4b8xu90h7,9817233883517681664,POINT( -84.3423355 41.26816799999996 8.0601425...
7,https://ufokn.org/id/urmi/dpk4b8xu90h7,9817233883517681664,POINT( -84.3423355 41.26816799999996 8.0601425...
8,https://ufokn.org/id/urmi/dpk4b8xu90h7,9817233883517681664,POINT( -84.3423355 41.26816799999996 8.0601425...
9,https://ufokn.org/id/urmi/dpk4b8xu90h7,9817233883517681664,POINT( -84.3423355 41.26816799999996 8.0601425...


In [50]:
rp1 = """
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX unit: <http://www.opengis.net/def/uom/OGC/1.0/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>

select ?s ?kwgres ?s2l13 where {
            ?s schema:additionalType "power"  .
            ?s schema:geo ?geo .        

            ?geo schema:identifier ?geoid .
            ?geoid schema:name "s2Level13" .
            ?geoid schema:value ?s2l13 .

            ?geo geo:asWKT ?buildingLocation .

            BIND("POINT(-84.3555 41.3175)"^^geo:wktLiteral AS ?centroid)
            BIND(geof:buffer(?centroid, 8.04672, unit:kilometer) AS ?searchArea)

            OPTIONAL {
                SERVICE <https://stko-kwg.geog.ucsb.edu/workbench/repositories/KWG> {
                    select * where {
                        ?kwgs ?kwgp ?s2l13 .
                        ?kwgres ?kwgp2 ?kwgs
                    }
                }
            }
}
"""

ep = "http://132.249.238.155/repositories/wenokn_ohio"

df = get_sparql_dataframe(ep, rp1)
df.head(10)

,s,kwgres,s2l13
0,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/Nie...,9817229210593263616
1,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/nat...,9817229210593263616
2,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/nat...,9817229210593263616
3,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/Ear...,9817229210593263616
4,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/NWZ...,9817229210593263616
5,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/cen...,9817229210593263616
6,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/noa...,9817229210593263616
7,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/dis...,9817229210593263616
8,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....,9817229210593263616
9,https://ufokn.org/id/urmi/dpk4bzkkghxv,http://stko-kwg.geog.ucsb.edu/lod/resource/smo...,9817229210593263616


In [68]:
rp1 = """
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX unit: <http://www.opengis.net/def/uom/OGC/1.0/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>

select DISTINCT ?s ?s2id ?buildingLocation
where {
    SERVICE <https://stko-kwg.geog.ucsb.edu/workbench/repositories/KWG> {
        select * where {
            kwgr:census.USCensusMSA.31000US19580.2019 kwg-ont:sfContains ?s2res .
            ?s2res kwg-ont:hasID ?s2res_id . 
        } 
    }
    BIND( ?s2res_id AS ?s2id)   

    ?s schema:additionalType "power" .
    ?s schema:geo ?geo .

    ?geo schema:identifier ?geoid .
    ?geoid schema:name "s2Level13" .
    ?geoid schema:value ?s2id .

    ?geo geo:asWKT ?buildingLocation .
} 
"""


ep = "http://132.249.238.155/repositories/wenokn_ohio"

df = get_sparql_dataframe(ep, rp1)
df.head(100)

,s,s2id,buildingLocation
